In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
# Get overlap params
mc_file_path = "../data/test_mc_df.xlsx"
mc_df = pd.read_excel(mc_file_path)
mc_df['params'] = mc_df['warehouse'] + "/" + mc_df['maker'] + "/" + mc_df['spec_name']+"/"+ mc_df['thickness'].astype(str)
mc_arr = mc_df['params'].unique()

fin_file_path = "../data/test_finish_df_Jul2022.xlsx"
fin_df = pd.read_excel(mc_file_path)
fin_df['params'] = fin_df['warehouse'] + "/" + fin_df['maker'] + "/" + fin_df['spec_name']+"/"+ fin_df['thickness'].astype(str)
fin_arr = fin_df['params'].unique()

# Find the intersection (overlapping values)
overlap = set(mc_arr) & set(fin_arr)

# Convert the result back to a list if needed
overlap_list = list(overlap)
len(overlap_list)

112

### DATA MANAGEMENT

In [3]:
## PARAMETER - CASE 0

PARAMS = {"warehouse": "HSC"
          ,"spec_name": "JSH440W-PO" # yeu cau chuan hoa du lieu OP - PO
          ,"thickness": 2
          ,"maker" : "POSCO-KR"
          ,"stock_ratio": { #get from app 
                    "limited": None
                    # "default": 2
                    # "user_setting": 4
                }
        #   ,"forecast_scenario": median
          }

margin_dict = { "HSC":
    {#save margin_dict in azure env, margin dictionary by warehouse
    "thickness_2.6": {
        "thickness": 2.6,
        "margin": 10
    },
    "thickness_2": {
        "thickness": 2.0,
        "margin": 8
    },
    "thickness_1.6": {
        "thickness": 1.6,
        "margin": 6
    }
    },
    "NQS":
    {#save margin_dict in azure env, margin dictionary by warehouse
    "thickness_2.6": {
        "thickness": 2.6,
        "margin": 10
    },
    "thickness_2": {
        "thickness": 2.0,
        "margin": 8
    },
    "thickness_1.6": {
        "thickness": 1.6,
        "margin": 6
    }
    }
}

# GET ALL PARAMS
MIN_MARGIN = margin_dict[PARAMS["warehouse"]][f"thickness_{PARAMS["thickness"]}"]["margin"]
print(f"MIN_MARGIN:{MIN_MARGIN}")

BOUND_KEY = next(iter(PARAMS['stock_ratio']))
BOUND_VALUE = PARAMS['stock_ratio'][BOUND_KEY]
print(f"BOUND_VALUE:{BOUND_VALUE}")

MIN_MARGIN:8
BOUND_VALUE:None


#### Input & Process Data

In [4]:
from process_df import filter_finish_df_to_dict, filter_by_params

file_path = "../data/test_finish_df_Jul2022.xlsx"
finish_key, value_columns = "order_id", ["width", "need_cut", f"upper_bound", "fc1", "fc2", "fc3"]
df = filter_by_params(file_path,PARAMS)
finish = filter_finish_df_to_dict(df, finish_key, value_columns, BOUND_VALUE)
finish
# nen loc list need cut voi so am truoc,
#Improve: uu tien thu solution voi FG co width lon truoc

{'F88': {'width': 390.0,
  'need_cut': 2036,
  'upper_bound': 4265.795,
  'fc1': 7432.65,
  'fc2': 4930.2,
  'fc3': 4706.1},
 'F87': {'width': 354.0,
  'need_cut': 9642,
  'upper_bound': 23399.847840000002,
  'fc1': 45859.4928,
  'fc2': 39600.1584,
  'fc3': 36639.6624},
 'F86': {'width': 269.0,
  'need_cut': 8500,
  'upper_bound': 13470.692319999998,
  'fc1': 16568.9744,
  'fc2': 23670.539200000003,
  'fc3': 20857.32},
 'F84': {'width': 215.0,
  'need_cut': 4300,
  'upper_bound': 5582.8426,
  'fc1': 4276.142,
  'fc2': 4074.3419999999996,
  'fc3': 3594.0579999999995},
 'F91': {'width': 70.0,
  'need_cut': 1200,
  'upper_bound': 1801.9351199999999,
  'fc1': 2006.4503999999997,
  'fc2': 1732.5911999999996,
  'fc3': 1603.0631999999998}}

In [5]:
total_needcut = sum(item['need_cut'] for item in finish.values())
print(f'Need cut {total_needcut}')

Need cut 25678


In [6]:
from process_df import filter_multi_stock_df_to_dict, filter_by_params
file_path = "../data/test_mc_df.xlsx"
stock_key, value_columns = "inventory_id", ['receiving_date','status',"width", "weight","qty"]
df = filter_by_params(file_path, PARAMS)
stocks = filter_multi_stock_df_to_dict(df, stock_key, value_columns,total_needcut)
print(len(stocks))
# list of stock >> list of need cut PO
# Improve: need rules to pick up stock to the pool

8


In [7]:
stocks

{'S438': {'receiving_date': '2022/07/18',
  'status': 'M:RAW MATERIAL',
  'width': 1219.0,
  'weight': 6180,
  'qty': 1},
 'S442': {'receiving_date': '2022/07/18',
  'status': 'M:RAW MATERIAL',
  'width': 1219.0,
  'weight': 6780,
  'qty': 1},
 'S440': {'receiving_date': '2022/07/18',
  'status': 'M:RAW MATERIAL',
  'width': 1219.0,
  'weight': 6820,
  'qty': 1},
 'S147': {'receiving_date': '2022/05/19',
  'status': 'M:RAW MATERIAL',
  'width': 1219.0,
  'weight': 7010,
  'qty': 1},
 'S144': {'receiving_date': '2022/05/19',
  'status': 'M:RAW MATERIAL',
  'width': 1219.0,
  'weight': 7390,
  'qty': 1},
 'S148': {'receiving_date': '2022/05/19',
  'status': 'M:RAW MATERIAL',
  'width': 1219.0,
  'weight': 7390,
  'qty': 1},
 'S145': {'receiving_date': '2022/05/19',
  'status': 'M:RAW MATERIAL',
  'width': 1219.0,
  'weight': 7400,
  'qty': 1},
 'S146': {'receiving_date': '2022/05/19',
  'status': 'M:RAW MATERIAL',
  'width': 1219.0,
  'weight': 7440,
  'qty': 1}}

## Operator to pick stocks - same width - ~ same weight